In [7]:
import numpy as np
import pandas as pd
import re
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

In [8]:
data = pd.read_pickle('../data/data_keep_57200.pkl')
train_df = pd.read_csv('../data/meinian_round1_train_20180408.csv')
test_df = pd.read_csv('../data/meinian_round1_test_a_20180409.csv')

# 通过 describe 查看特征的类型，标准差，修正类型和异常数据
train_df['收缩压'] = pd.to_numeric(train_df['收缩压'], errors='coerce')
train_df['舒张压'] = pd.to_numeric(train_df['舒张压'], errors='coerce')
train_df.loc[22712, '血清甘油三酯'] = 7.75
train_df['血清甘油三酯'] = pd.to_numeric(train_df['血清甘油三酯'], errors='coerce')
# train_df['舒张压'].sort_values(ascending=False)[:5]
train_df.loc[22357, '舒张压'] = np.nan
train_df.loc[29394, '舒张压'] = np.nan
train_df.loc[29394, '收缩压'] = np.nan
train_df['血清低密度脂蛋白'][train_df['血清低密度脂蛋白']<0] = 0

In [9]:
data['vid'] = data.index

In [10]:
# 合并数据
merged_train_df = pd.merge(train_df, data, on='vid', sort=False)
merged_test_df = pd.merge(test_df, data, on='vid', sort=False)
combine = [merged_train_df, merged_test_df]

In [11]:
print(merged_train_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38199 entries, 0 to 38198
Columns: 1141 entries, vid to Y79001
dtypes: float64(5), object(1136)
memory usage: 332.8+ MB
None


In [12]:
# 数值成分字典
feature_num_prop = {}

def get_num_prop(data_col):
    num_counts = data_col.astype(
        str).str.match(r'^(-?\d+)(\.\d+)?$').sum()
    na_counts = data_col.isna().sum()
    return num_counts / (data_col.shape[0] - na_counts)

for col in merged_train_df.columns.values:
    feature_num_prop[col] = get_num_prop(merged_train_df[col])

# label = numerical_feature[0:5]        
# numerical_feature = numerical_feature[5:]
# print('numerical feature count: %s' %len(numerical_feature))
# print(numerical_feature)

In [13]:
num_more_than_05 = [k for k,v in feature_num_prop.items() if v > 0.5 ]
num_more_than_01 = [k for k,v in feature_num_prop.items() if 0.1 < v <= 0.5 ]
num_less_than_01 = [k for k,v in feature_num_prop.items() if v <= 0.1 ]

In [14]:
len(num_more_than_05),len(num_more_than_01),len(num_less_than_01)

(662, 29, 450)

In [ ]:
'0124' in num_less_than_01

In [ ]:
def search_non_numeric(data):
    if not re.search(r'^(-?\d+)(\.\d+)?$', data) and data!='nan':
        non_numeric.append(data)

# non_numeric = []
# # applymap 会有问题，第一列会操作两次
# for col in numerical_feature:
#     non_numeric.append('----'+col+'----')
#     temp = merged_train_df[col].astype('str').apply(search_non_numeric)
# for col in numerical_feature:
#     non_numeric.append('----'+col+'----')
#     temp = merged_test_df[col].astype('str').apply(search_non_numeric)

In [ ]:
# len(non_numeric)

In [ ]:
# with open('mix_in_numeric.txt', 'w') as f:
#     for t in non_numeric:
#         f.write(t+'\n')

In [ ]:
def print_non_num(feature_series):
    print(feature_series[feature_series.str.match(r'^(-?\d+)(\.\d+)?$')==False])

In [ ]:
def convert_mixed_num(data):
    data = data.strip()
    special_cases = ['未见','阴性']
    try:
        ret = float(data)
        return ret if data >=0 else np.nan
    except:
        if data in special_cases:
            return 0
        all_match = re.findall(r'\d+\.?\d*', data) # 注意：不带负号
        if all_match:
            all_list = [float(i) for i in all_match]
            return sum(all_list)/len(all_list)    # 取均值
        else:
            return np.nan

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

test = merged_train_df['269023'].astype(
        'str').apply(convert_mixed_num).dropna()
sns.distplot(test)
plt.show()

In [ ]:
merged_train_df.loc[32230,'1850'] = 3.89
merged_train_df.loc[[2527,3027],'192'] = 16.07, 12.01
merged_train_df.loc[3163,'193'] = np.nan
merged_train_df.loc[6055,'2333'] = 5.0    # 多了小数点
merged_train_df.loc[5085,'269013']    # 未见，映射成0
merged_train_df.loc[[8551,8840,9072,9309],'3193'] = '>=1.030'
merged_test_df.loc[2327, '3193'] = '>=1.030'
merged_test_df.loc[2327, '1840'] = '<=5.0'

In [ ]:
merged_train_df.loc[21196,'2405'] = np.nan  # 异常大
merged_train_df.loc[33729,'0424'] = np.nan
merged_train_df.loc[21196,'2403'] = np.nan

In [ ]:
# RF 得到的特征重要性, 暂时不要 drop
# low_importance = ['269024', '979013', '979018', '1325', '979014', '1326']

for df in combine:
    df[numerical_feature] = df[numerical_feature].astype(
    'str').applymap(convert_mixed_num)
    # to_fill = df[numerical_feature].median()
    # df[numerical_feature] = preprocessing.robust_scale(df[numerical_feature].fillna(to_fill))
    # df.drop(columns=low_importance, inplace=True)    # 去掉不重要的特征

need_log1p = ['100007', '1117', '1127', '1814', '1815', '183']
for col in need_log1p:
    for df in combine:
        df[col] = np.log1p(df[col])

In [ ]:
numerical_feature = [i for i in numerical_feature if i not in low_importance]

In [ ]:
label_numerical_df = merged_train_df[label + numerical_feature]

In [ ]:
# corr = label_numerical_df.corr()
# sns.pairplot(corr)

In [ ]:
# plt.show()

In [ ]:
merged_test_df.describe()

In [ ]:
merged_train_df['2403'].dropna().sort_values().tail()

In [ ]:
# test = merged_train_df['100007'].dropna()
test = merged_train_df['2174']
sns.distplot(test.dropna())
plt.show()

In [ ]:
# %matplotlib inline
# fig = plt.gcf()
# fig.set_size_inches(18, 18)
# sns.heatmap(merged_train_df[label+numerical_feature].corr())
# plt.show()

In [ ]:
merged_train_df.info()

In [ ]:
merged_train_df[numerical_feature].describe()

In [ ]:
# abnormal_num = []
# for col in numerical_feature:
#     if merged_train_df[col].max() > 1000:
#         abnormal_num.append(col)
# print(abnormal_num)

In [ ]:
merged_train_df.info()

In [ ]:
merged_train_df.describe(include='O')

In [ ]:
# 这行开始


In [ ]:
non_numerical_feature = merged_train_df.describe(include='O').columns.values[1:]
len(non_numerical_feature)

In [ ]:
non_numerical_df = merged_train_df[non_numerical_feature]
non_numerical_desc = non_numerical_df.describe()

In [ ]:
unique_less_than_100 = non_numerical_desc.iloc[1,:][non_numerical_desc.iloc[1,:] < 100].index.values

In [ ]:
len(unique_less_than_100)

In [ ]:
unique_less_than_100

In [2]:
# 方便实验，重新载入
merged_train_df = pd.read_pickle('../data/data_train_num.pkl')
merged_test_df = pd.read_pickle('../data/data_test_num.pkl')
combine = [merged_train_df, merged_test_df]

In [ ]:
# 去掉前后空白
for col in non_numerical_feature:
    merged_train_df.loc[:, col] = merged_train_df.loc[:, col].str.strip()
    merged_test_df.loc[:, col] = merged_test_df.loc[:, col].str.strip()
# sns.countplot(y='0124', data=merged_train_df, palette="Greens_d");·`

In [ ]:
def converter(pat):
    def convert(data):
        if data == data:
            if re.search(pat, data):
                return 0
            else:
                return 1
        return data
    return convert

In [ ]:
# 测试转换之后结果
for df in combine:
    df['0405'] = df['0405'].apply(converter(r'(无|未)')).astype('category')

In [ ]:
for df in combine:
    df['1315'] = df['1315'].apply(converter(r'(未|正常)')).astype('category')

In [ ]:
for df in combine:
    df['0407'] = df['0407'].apply(converter(r'(未|弃)')).astype('category')

In [ ]:
for df in combine:    
    df['0420'] = df['0420'].apply(converter(r'(未|正常)')).astype('category')

In [ ]:
def convert_0421(data):
    if data == data:
        normal = ['整齐','齐','正常','整','整齐;整齐','齐;齐','未见异常']
        if data in normal:
            return 0
        elif re.search(r'早搏',data):
            return 1
        elif re.search(r'(不齐|过|窦性)',data):
            return 2
        elif re.search(r'房颤',data):
            return 3
        elif re.search(r'齐',data):
            return 0
    return np.nan
for df in combine:    
    df['0421'] = df['0421'].apply(convert_0421).astype('category')

In [ ]:
pd.set_option('display.max_rows', 100)

In [5]:
merged_test_df['0124']

0                  None
1                  None
2                  None
3                  None
4       输尿管上段、膀胱壁间段无扩张。
5                  None
6                  None
7                  None
8                  None
9                  None
10                 None
11                 None
12         双侧输尿管未见明显扩张。
13                 None
14                 None
15                 None
16                 None
17                 None
18                 None
19                 None
20                 None
21      输尿管上段、膀胱壁间段无扩张。
22                 None
23      输尿管上段、膀胱壁间段无扩张。
24                 None
             ...       
9513    输尿管上段、膀胱壁间段无扩张。
9514               None
9515               None
9516               None
9517    输尿管上段、膀胱壁间段无扩张。
9518    输尿管上段、膀胱壁间段无扩张。
9519               None
9520    输尿管上段、膀胱壁间段无扩张。
9521    输尿管上段、膀胱壁间段无扩张。
9522               None
9523               None
9524               None
9525    输尿管上段、膀胱壁间段无扩张。
9526               None
9527               None
9528            

In [ ]:
merged_train_df['3485'].value_counts()

In [ ]:
merged_train_df.groupby('3400').mean().iloc[:,0:5].sort_values(by='收缩压',ascending=False)

In [ ]:
merged_train_df.describe(include='category')

In [ ]:
merged_train_df['0405'].value_counts().index[0]

In [ ]:
cate_feature = merged_train_df.describe(include='category').columns.values.tolist()
most_cate = {}
for col in cate_feature:
    most_cate[col] = merged_train_df[col].value_counts().index[0]

In [ ]:
most_cate